<a href="https://colab.research.google.com/github/FunzanRoy/AI/blob/main/Classic_Chinese_Inference_with_Adapter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 利用PEFT微調過後的Adapter，進行白話文與文言文轉換


In [1]:
!pip install transformers datasets torch bitsandbytes peft accelerate nvidia-ml-py3 wandb trl flash-attn

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19173 sha256=c725371b6de847fbcfec93bc35b724a2e57a06314c4fda0d424578490b700817
  Stored in directory: /root/.cache/pip/wheels/5c/d8/c0/46899f8be7a75a2ffd197a23c8797700ea858b9b34819fbf9e
 

## 使用量化之後，載入Adapter的方式
* 引入套件，這次要包括量化用的套件BitsAndBytes的Config
* 設定量化的參數，和訓練時要一模一樣
    * `load_in_4bit=True`在載入模型時將模型量化為 4 位元
    * `bnb_4bit_use_double_quant=True`使用嵌套量化方案來量化已經量化的權重
    * `bnb_4bit_quant_type="nf4"`為對從常態分佈初始化的權重使用特殊的 4 位元資料類型
    * `bnb_4bit_compute_dtype=torch.bfloat16`使用 bfloat16 來加快計算速度
* 此時載入基礎模型時需要設定`config`
* 後面一樣將Adapter載入，並且指定`tokenizer`和`device`

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = "zake7749/gemma-2-2b-it-chinese-kyara-dpo"
peft_model = "huchiahsi/peft-model-repo"
model = AutoModelForCausalLM.from_pretrained(base_model,
                                                config=bnb_config,
                                                device_map="auto")
model = PeftModel.from_pretrained(model, peft_model)
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = model.to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

adapter_config.json:   0%|          | 0.00/814 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/332M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [3]:
import torch
inputs = tokenizer("台北市很漂亮，文言文怎麼說？", return_tensors="pt")

with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=222)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


台北市很漂亮，文言文怎麼說？
翻譯成現代文：
昔者夏，禹欲治海，見天下方平，命臣從之，成太白高升焉。 
